In [1]:
# @title
!pip install langchain-gigachat langchain-chroma python-dotenv

INFO: pip is looking at multiple versions of langchain-chroma to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 95.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.9/69.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.8/450.8 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 90.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 13.0 MB/s eta 0:00:0

In [2]:
import os
from google.colab import userdata
from langchain_gigachat.chat_models import GigaChat

In [3]:
auth = userdata.get('SBER_AUTH')

llm = GigaChat(credentials= auth, verify_ssl_certs = False)

In [ ]:
from langchain.schema import HumanMessage

question = "Ответь кратко. Какой плащ был у Понтия Пилата?"
llm([HumanMessage(content=question)]).content[0:200]

<ipython-input-4-991c0d0b209a>:4: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm([HumanMessage(content=question)]).content[0:200]


'У Понтия Пилата не было конкретного плаща, который можно было бы выделить как его отличительный элемент одежды. В библейских рассказах он чаще всего описан просто как римский префект Иудеи в обычной о'

In [ ]:
!pip install langchain

In [ ]:
!pip install langchain-community langchain-core
from langchain.document_loaders import TextLoader
#from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.8 MB/s eta 0:00:00


In [ ]:
loader = TextLoader("./Master_i_Margarita.txt")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
documents = text_splitter.split_documents(documents)
print(f"Total documents: {len(documents)}")

Total documents: 65


In [ ]:
from chromadb.config import Settings
from langchain_gigachat.embeddings.gigachat import GigaChatEmbeddings
from langchain_chroma import Chroma

embeddings = GigaChatEmbeddings(
    credentials = auth, verify_ssl_certs=False
)

db = Chroma.from_documents(
    documents,
    embeddings,
    client_settings=Settings(anonymized_telemetry=False),
)

In [ ]:
question = "Какой плащ был у Понтия Пилата?"
docs = db.similarity_search(question, k=4)
len(docs)

4

In [ ]:
print(f"... {str(docs[1])} ...")

... page_content='В белом плаще с кровавым подбоем, шаркающей кавалерийской походкой, ранним утром четырнадцатого числа весеннего месяца нисана в крытую колоннаду между двумя крыльями дворца ирода великого вышел прокуратор Иудеи Понтий Пилат.
Более всего на свете прокуратор ненавидел запах розового масла, и все теперь предвещало нехороший день, так как запах этот начал преследовать прокуратора с рассвета. Прокуратору казалось, что розовый запах источают кипарисы и пальмы в саду, что к запаху кожи и конвоя примешивается проклятая розовая струя. От флигелей в тылу дворца, где расположилась пришедшая с прокуратором в Ершалаим первая когорта двенадцатого молниеносного легиона, заносило дымком в колоннаду через верхнюю площадку сада, и к горьковатому дыму, свидетельствовавшему о том, что кашевары в кентуриях начали готовить обед, примешивался все тот же жирный розовый дух. О боги, боги, за что вы наказываете меня?' metadata={'source': './Master_i_Margarita.txt'} ...


In [ ]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(llm, retriever=db.as_retriever())

In [ ]:
qa_chain({"query": question})

<ipython-input-15-ee0d9041a533>:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  qa_chain({"query": question})


{'query': 'Какой плащ был у Понтия Пилата?',
 'result': 'У Понтия Пилата был белый плащ с кровавым подбоем.'}

In [ ]:
qa_chain({"query": "Какая трость была у Воланда?"})

{'query': 'Какая трость была у Воланда?',
 'result': 'У Воланда была трость с черным набалдашником в виде головы пуделя.'}

In [ ]:
qa_chain({"query": "что не смогли купить герои романа на Патриарших?"})

{'query': 'что не смогли купить герои романа на Патриарших?',
 'result': 'Герои романа не смогли купить пиво, так как его пообещали привести только вечером. Также им не удалось приобрести нарзан, поскольку его не оказалось в будочке. Единственный напиток, который они нашли, была теплая абрикосовая вода.'}

#**Работа с PDF документами**

In [ ]:
!pip install pypdf langchain-chroma --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.4/303.4 kB 16.0 MB/s eta 0:00:00


In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("http://www.unn.ru/books/met_files/Alekseev.pdf")

docs = loader.load()
print(len(docs))

139


In [ ]:
print(docs[0].page_content[0:100])
print(docs[0].metadata)

МИНИСТЕРСТВО ОБРАЗОВАНИЯ И НАУКИ  
РОССИЙСКОЙ ФЕДЕРАЦИИ 
 
Нижегородский государственный университет
{'producer': 'Microsoft® Office Word 2007', 'creator': 'Microsoft® Office Word 2007', 'creationdate': '2017-12-11T16:46:24+03:00', 'author': 'АВЕ', 'moddate': '2017-12-11T16:46:24+03:00', 'source': 'http://www.unn.ru/books/met_files/Alekseev.pdf', 'total_pages': 139, 'page': 0, 'page_label': '1'}


In [ ]:
from langchain_gigachat.chat_models import GigaChat
from langchain.schema import HumanMessage

giga = GigaChat(
    credentials=auth,
    model="GigaChat-Pro",
    verify_ssl_certs=False,
    timeout=1200,
)

In [ ]:
from langchain_chroma import Chroma
from langchain_gigachat.embeddings.gigachat import GigaChatEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

embeddings = GigaChatEmbeddings(
    credentials=auth, verify_ssl_certs=False
)


text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

retriever = vectorstore.as_retriever()

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "Ты должен ответить на вопрос пользователя с использованием данных из книги.\n"
    "Отвечай коротко, не более 2-3 предложений.\n"
    "Вот части книги контекст для ответа:"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(giga, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

Оригинал: Теорема 5.7 (об эйлеровом цикле). Эйлеров цикл в связном графе существует тогда и только тогда, когда степени всех вершин этого графа четны.

In [ ]:
q = "О чем говорит теорема об эйлеровом цикле?"
pure = giga([HumanMessage(content=q)]).content
print(f"Pure: {pure}")

rag = rag_chain.invoke({"input": q})["answer"]
print(f"RAG: {rag}")

Pure: Теорема об Эйлеровом цикле (или теорема Эйлера) касается графов и их свойств. Она утверждает следующее:

**Теорема:** Граф имеет замкнутый Эйлеров цикл тогда и только тогда, когда он связен и все его вершины имеют четную степень.

### Пояснение:
- **Эйлеров цикл** — это путь в графе, который проходит через каждое ребро ровно один раз и возвращается в исходную вершину.
- **Связность** означает, что между любой парой вершин существует хотя бы один путь.
- **Четная степень** — каждая вершина должна иметь чётное количество рёбер, инцидентных ей.

### Пример:
Рассмотрим пример с известным классическим графом — картой улиц Кёнигсберга. Этот город имел семь мостов, соединяющих разные части города. Задача состояла в том, чтобы пройти по каждому мосту ровно один раз и вернуться обратно. Это оказалось невозможным, так как не все вершины имели чётную степень (все они были нечётными).

Таким образом, теорема Эйлера даёт необходимое и достаточное условие для существования такого цикла. Если г

Оригинал:  Оказывается, присутствие в графе нечетных циклов (т.е. циклов нечетной длины) – единственная причина, по которой граф не является двудольным.

In [ ]:
q = "Короткий ответ: По каким причинам граф не будет двудольным? "
pure = giga([HumanMessage(content=q)]).content
print(f"Pure: {pure}\n")

rag = rag_chain.invoke({"input": q})["answer"]
print(f"RAG: {rag}")

Pure: Граф не будет двудольным, если в нём существует цикл длиной $3$ (или более), все вершины которого попарно смежны. Это связано с тем, что в двудольном графе любые две вершины одного цвета не могут быть соединены ребром, а наличие такого цикла означает, что есть как минимум две вершины одного цвета, которые должны быть связаны ребром.

RAG: Граф не будет двудольным, если он содержит нечетные циклы. Это единственная причина, по которой граф не является двудольным. Теорема 5.14 подтверждает это утверждение.


Каркас (применяются также термины остов, остовное дерево) связного графа – это дерево, являющееся подграфом этого графа и содержащего все его вершины.

In [ ]:
q = "Дай определение: что такое каркас?"
pure = giga([HumanMessage(content=q)]).content
print(f"Pure: {pure}\n")

rag = rag_chain.invoke({"input": q})["answer"]
print(f"RAG: {rag}")

Pure: Каркас – это основная несущая конструкция здания, сооружения или изделия, которая обеспечивает его прочность, жесткость и устойчивость. Обычно он состоит из различных элементов (балок, колонн, ферм, рам), которые воспринимают нагрузки и передают их на фундамент или другие опорные конструкции. В зависимости от материала каркасы могут быть деревянными, металлическими, железобетонными или комбинированными.

RAG: Каркас (или остов, остовное дерево) связного графа – это дерево, являющееся подграфом этого графа и содержащего все его вершины.
